In [6]:
from textgrad import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.loss import TextLoss

import sys, os
root_dir = os.getcwd()
parent_dir = os.path.dirname(root_dir)
sys.path.append(parent_dir)
from llm_agents.llm.openai_model import AnyOpenAILLM
import platformdirs
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from textgrad.engine.base import EngineLM, CachedEngine



x = Variable("A sntence with a typo", role_description="The input sentence", requires_grad=True)



class ChatOpenAI(EngineLM, CachedEngine):
    DEFAULT_SYSTEM_PROMPT = "You are a helpful, creative, and smart assistant."

    def __init__(
        self,
        model_string="chatgpt-4k",
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        **kwargs):
        """
        :param model_string:
        :param system_prompt:
        """
        root = platformdirs.user_cache_dir("textgrad")
        cache_path = os.path.join(root, f"cache_openai_{model_string}.db")

        super().__init__(cache_path=cache_path)

        self.system_prompt = system_prompt
        self.client = AnyOpenAILLM( model_string)
        self.model_string = model_string



    def generate(
        self, prompt, system_prompt=None, temperature=0, max_tokens=2000, top_p=0.99
    ):

        sys_prompt_arg = system_prompt if system_prompt else self.system_prompt

        cache_or_none = self._check_cache(sys_prompt_arg + prompt)
        if cache_or_none is not None:
            return cache_or_none

        response = self.client(prompt, sys_prompt_arg,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
        )        

        response = response.text[0]
        self._save_cache(sys_prompt_arg + prompt, response)
        return response

    @retry(wait=wait_random_exponential(min=1, max=5), stop=stop_after_attempt(5))
    def __call__(self, prompt, **kwargs):
        return self.generate(prompt, **kwargs)

engine = ChatOpenAI("chatgpt-4k") # "chatgpt-16k", "gpt4-short", "gpt-3.5-turbo-0125"
# engine.generate("Generate one good word?")

In [12]:
system_prompt = Variable("Evaluate the correctness of this sentence", role_description="The system prompt")
loss = TextLoss(system_prompt, engine=engine)

In [13]:
optimizer = TextualGradientDescent(parameters=[x], engine=engine)


In [14]:
l = loss(x)

In [17]:
x

Variable(value=A sntence with a typo, role=The input sentence, grads=)

In [15]:
l

Variable(value=The sentence is incorrect due to the typo., role=response from the language model, grads=)

In [ ]:
l.backward(engine)
optimizer.step()